# **Condiciones de Terminación en AutoGen**

En la sección anterior vimos cómo definir agentes y organizarlos en equipos para resolver tareas. Sin embargo, una ejecución puede continuar indefinidamente si no se le indica cuándo detenerse. Para esto, **AutoGen proporciona condiciones de terminación (termination conditions)**, que nos permiten definir criterios para detener una ejecución de manera automática.

## **🔹 ¿Qué es una condición de terminación?**

Una **condición de terminación** es una función que recibe una secuencia de mensajes (`AgentEvent` o `ChatMessage`) y decide si el equipo debe detenerse o continuar.  
Si se cumple la condición, devuelve un **`StopMessage`**. Si no, devuelve `None`.

📌 **Características clave**:

- Son **estado-dependientes**, pero **se reinician automáticamente** después de cada ejecución (`run()` o `run_stream()`).
- Se pueden **combinar** usando los operadores `AND (&)` y `OR (|)`.
- En **equipos multiagente**, la condición se evalúa **después de cada respuesta de un agente**

---
## **Tipos de Condiciones de Terminación**

AutoGen proporciona varias condiciones de terminación predefinidas. Aquí tienes un resumen:

|**Condición**|**Descripción**|
|---|---|
|**MaxMessageTermination**|Detiene la ejecución cuando se alcanza un número máximo de mensajes.|
|**TextMentionTermination**|Detiene la ejecución cuando se menciona una palabra específica (ejemplo: `"TERMINATE"`).|
|**TokenUsageTermination**|Detiene la ejecución cuando se usa un cierto número de tokens.|
|**TimeoutTermination**|Detiene la ejecución después de un tiempo límite en segundos.|
|**HandoffTermination**|Se detiene cuando un agente transfiere el control a otro (ejemplo: usuario humano).|
|**SourceMatchTermination**|Se detiene cuando un agente específico responde.|
|**ExternalTermination**|Permite detener la ejecución desde fuera del equipo (por ejemplo, desde una UI).|
|**StopMessageTermination**|Se detiene cuando un agente produce un `StopMessage`.|

---

## **🔹 Ejemplo 1: Uso de `MaxMessageTermination`**

Creamos un equipo con un **asistente principal (`primary_agent`)** que genera contenido y un **crítico (`critic_agent`)** que da feedback.  
Se detendrá después de **3 mensajes**.

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage
import os


model_client = AzureOpenAIChatCompletionClient(
    azure_deployment= "gpt-4o-mini", 
    model="gpt-4o-mini",  
    api_version=os.environ["OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    

)

result = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None


In [4]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os


# Agente principal (genera contenido)
primary_agent = AssistantAgent("primary", model_client=model_client, system_message="You are a helpful AI assistant.")

# Agente crítico (da feedback)
critic_agent = AssistantAgent("critic", model_client=model_client, system_message="Provide constructive feedback.")

# Condición de terminación: máximo 3 mensajes
max_msg_termination = MaxMessageTermination(max_messages=3)

# Crear el equipo
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# Ejecutar el equipo

await (Console(team.run_stream(task="Escribe un haiku sobre el clima en París.")))


---------- user ----------
Escribe un haiku sobre el clima en París.
---------- primary ----------
Nubes susurran,  
lluvia danza en el suelo,  
París se abraza.
---------- critic ----------
¡Excelente haiku! Capturas muy bien la atmósfera de París con la imagen de las nubes y la lluvia. La personificación de la lluvia como una danza es especialmente evocadora. Para que el haiku se sienta aún más completo, podrías considerar incluir un elemento que conecte aún más con el ambiente urbano de la ciudad, como un monumento o una actividad cotidiana. Pero en general, es una hermosa representación del clima y la esencia de París. ¡Buen trabajo!


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Escribe un haiku sobre el clima en París.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=22), content='Nubes susurran,  \nlluvia danza en el suelo,  \nParís se abraza.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=55, completion_tokens=100), content='¡Excelente haiku! Capturas muy bien la atmósfera de París con la imagen de las nubes y la lluvia. La personificación de la lluvia como una danza es especialmente evocadora. Para que el haiku se sienta aún más completo, podrías considerar incluir un elemento que conecte aún más con el ambiente urbano de la ciudad, como un monumento o una actividad cotidiana. Pero en general, es una hermosa representación del clima y la esencia de París. ¡Buen trabajo!', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

## **🔹 Ejemplo 2: Uso de `TextMentionTermination`**

En este caso, la conversación **se detiene cuando el crítico responde con `"APPROVE"`**. 

In [4]:

import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os


model_client = AzureOpenAIChatCompletionClient(
    azure_deployment= "gpt-4o-mini", 
    model="gpt-4o-mini",  
    api_version=os.environ["OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# Agente principal (genera contenido)
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant."
)

# Agente crítico (da feedback y aprueba)
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. If the haiku follows the correct structure, respond ONLY with 'APPROVE'."
)

# Condición de terminación: detener cuando el crítico mencione "APPROVE"
text_termination = TextMentionTermination("APPROVE")

# Crear el equipo
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

# Ejecutar el equipo

await (Console(team.run_stream(task="Escribe un haiku sobre el clima en París.")))



---------- user ----------
Escribe un haiku sobre el clima en París.
---------- primary ----------
Nubes danzan hoy,  
la brisa suave acaricia,  
París susurra.
---------- critic ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Escribe un haiku sobre el clima en París.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=20), content='Nubes danzan hoy,  \nla brisa suave acaricia,  \nParís susurra.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=3), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

## **🔹 Ejemplo 3: Combinar múltiples condiciones**

Podemos **combinar condiciones** con los operadores **AND (`&`)** y **OR (`|`)**:

|**Operador**|**Significado**|
|---|---|
|`cond1|cond2`|
|`cond1 & cond2`|**Se detiene solo si ambas condiciones se cumplen** (AND).|

💡 **Ejemplo: Detener si se alcanzan 10 mensajes o si el crítico aprueba**:

In [6]:
from autogen_agentchat.conditions import MaxMessageTermination

# Combinar condiciones con OR (|) → Se detiene si se cumple alguna
combined_termination = MaxMessageTermination(max_messages=10) | TextMentionTermination("APPROVE")

# Crear el equipo con la condición combinada
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Ejecutar el equipo
await (Console(team.run_stream(task="Escribe un haiku sobre el clima en París.")))


---------- user ----------
Escribe un haiku sobre el clima en París.
---------- primary ----------
Lluvia en el aire,  
el río canta y brilla,  
París se despide.
---------- critic ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Escribe un haiku sobre el clima en París.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=72, completion_tokens=22), content='Lluvia en el aire,  \nel río canta y brilla,  \nParís se despide.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=3), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

## **🔹 Ejemplo 4: Detener la ejecución desde el exterior (`ExternalTermination`)**

A veces queremos **detener manualmente** el equipo (por ejemplo, en una UI con un botón "STOP").  
Para esto, usamos `ExternalTermination`. 

In [ ]:
from autogen_agentchat.conditions import ExternalTermination

# Crear una condición de terminación externa
external_termination = ExternalTermination()

# Crear el equipo con la condición externa
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=external_termination)

# Iniciar la ejecución en segundo plano
import asyncio
task = asyncio.create_task(Console(team.run_stream(task="Escribe un poema corto.")))

# Simular una pausa y detener manualmente la ejecución
import time
time.sleep(5)
external_termination.set()  # 🔴 DETENER EJECUCIÓN

# Esperar a que termine la tarea
await(task)


RuntimeError: This event loop is already running